In [1]:
import pyabf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import pathlib
import pickle

In [2]:
# read final csv that has the ouput of metadata gathering process
csv_meta_save_path = 'output_files/cell_final_raw_meta_df.csv'
cell_final_raw_meta_df = pd.read_csv(open(csv_meta_save_path, 'rb')) 

In [3]:
def summarize_cell_ephys_features(lsa_results):
    hero_keep_features = ['adapt', 'avg_rate', 'first_isi', 'isi_cv', 'latency', 'mean_isi', 'median_isi', 
                          'stim_amp']
    rheo_keep_features = ['threshold_v', 'peak_v', 'trough_v', 
     'fast_trough_v', 'adp_v', 'width', 'upstroke_downstroke_ratio']
    
    overall_cell_keep_features = ['v_baseline', 'rheobase_i', 'fi_fit_slope', 
                                  'sag', 'vm_for_sag', 'input_resistance', 'tau']
    
    hero_small_dict = lsa_results['hero_sweep'][hero_keep_features]
    rheobase_sweep_index = lsa_results['rheobase_sweep'].name
    rheobase_sweep = lsa_results['spikes_set'][rheobase_sweep_index].iloc[0]
    rheo_spike_small_dict = rheobase_sweep[rheo_keep_features]
    spike_comb_dict = {**hero_small_dict, **rheo_spike_small_dict}

    overall_cell_features = {x: lsa_results[x] for x in overall_cell_keep_features if x in lsa_results}
    final_cell_feature_dict = {**spike_comb_dict, **overall_cell_features}
    return(final_cell_feature_dict)

In [4]:
import pandas as pd
from ipfx import time_series_utils as tsu
from ipfx.sweep import Sweep, SweepSet

def get_rheo_spike_waveform_features(sweep_set, lsa_results):
    rheo_sweep_ind = lsa_results['rheobase_sweep'].name
    rheo_sweep = sweep_set.sweeps[rheo_sweep_ind]
    
    final_ap_time = .010 # 10 ms in seconds
    
    threshold_index = int(lsa_results['spikes_set'][rheo_sweep_ind]['threshold_index'][0])
    trough_index = int(lsa_results['spikes_set'][rheo_sweep_ind]['fast_trough_index'][0])
    
    print(rheo_sweep.sampling_rate)
    final_ap_time_index = int(threshold_index + rheo_sweep.sampling_rate * final_ap_time)
    
    dvdt = tsu.calculate_dvdt(rheo_sweep.v, rheo_sweep.t, filter = 2)
    
    ap_v = rheo_sweep.v[threshold_index:final_ap_time_index]
    ap_t = rheo_sweep.t[threshold_index:final_ap_time_index] - rheo_sweep.t[threshold_index]
    
#     df = pd.DataFrame(data = {'t':ap_t, 'v': ap_v})
    
#     # upsample time series if needed
#     df[ 't' ] = pd.to_datetime(df['t'], unit = 's')
    
    ap_dvdt = dvdt[threshold_index:final_ap_time_index]
    
    df = pd.DataFrame(data = {'t':ap_t, 'v': ap_v, 'dvdt' : ap_dvdt})
    
    return df

In [5]:
# go through lsa feature objects and load features and then write to csv
use_meta_df = cell_final_raw_meta_df.query('valid_stim == True')
cell_fail_list = list()
cell_feature_list = list()
ap_cell_feature_list = list()

update_existing = False
for abf_file_name in use_meta_df.cell_id:
    print(abf_file_name)
    fn = "lsa_features/" + abf_file_name + '.p'
    file = pathlib.Path(fn)
    if file.exists():
        with open(fn, 'rb') as fp:
            lsa_results = pickle.load(fp)
    fn = "sweep_sets/" + abf_file_name + '.p'
    file = pathlib.Path(fn)
    if file.exists():
        with open(fn, 'rb') as fp:
            sweep_set = pickle.load(fp)
    ap_waveform_features = get_rheo_spike_waveform_features(sweep_set, lsa_results)
    ap_waveform_features['cell_id'] = abf_file_name
    ap_cell_feature_list.append(ap_waveform_features)
    
    curr_cell_sweep_df = lsa_results['sweeps']
    curr_cell_sweep_df['cell_id'] = abf_file_name
    curr_cell_sweep_df['norm_stim_amp'] = curr_cell_sweep_df['stim_amp'] / lsa_results['rheobase_i']
    cell_feature_list.append(curr_cell_sweep_df)
fi_features = pd.concat(cell_feature_list)
fi_features_csv_path = 'output_files/fi_curves.csv'
fi_features.to_csv(fi_features_csv_path)

ap_features = pd.concat(ap_cell_feature_list)
ap_features_csv_path = 'output_files/ap_features.csv'
ap_features.to_csv(ap_features_csv_path)
        

13n05011.abf
10000
13n21007.abf


/Users/stripathy/anaconda3/lib/python3.7/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


10000
13d02004.abf
10000
13d02022.abf
10000
15o08007.abf
9803
13d03005.abf
10000
13d03007.abf
10000
13d03008.abf
10000
13d03029.abf
10000
13d03040.abf
10000
20131211_600_1_0190.abf
10000
20131211_600_1_0184.abf
10000
20131211_600_1_0160.abf
10000
20131211_600_1_0108.abf
10000
20131211_600_1_0042.abf
10000
20131211_600_1_0067.abf
10000
20131211_600_1_0066.abf
10000
20131211_600_1_0002.abf
10000
20131211_600_1_0081.abf
10000
20140127_600_1_0118.abf
10000
20140127_600_1_0158.abf
10000
14304311.abf
10000
14304300.abf
10000
14317320.abf
10000
14317348.abf
10000
14317300.abf
10000
14318308.abf
10000
14408304.abf
10000
14424000.abf
25000
14424329.abf
25000
14424335.abf
10000
14515424.abf
25000
14515300.abf
25000
14515323.abf
10000
14515349.abf
10000
14515372.abf
10000
14515374.abf
10000
14520332.abf
25000
14520300.abf
25000
14605300.abf
25000
14605321.abf
25000
14605325.abf
25000
14605338.abf
25000
14605349.abf
25000
14617300.abf
25000
14617312.abf
25000
14624359.abf
10000
14624325.abf
25000


In [35]:
fi_features

,adapt,avg_rate,first_isi,isi_cv,latency,mean_isi,median_isi,peak_deflect,sag,stim_amp,v_baseline,cell_id,norm_stim_amp
0,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,"(-94.45801, 1770)",0.074268,-400.000006,-54.411869,13n05011.abf,-8.000000
1,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,"(-88.96484, 3827)",0.067792,-350.000005,-53.585289,13n05011.abf,-7.000000
2,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,"(-85.7605, 2583)",0.027194,-300.000004,-51.284962,13n05011.abf,-6.000000
3,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,"(-82.96509, 2229)",0.052340,-250.000004,-54.780979,13n05011.abf,-5.000000
4,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,"(-76.99585, 2898)",0.063671,-200.000003,-52.602428,13n05011.abf,-4.000000
5,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,"(-72.75391, 3687)",0.111978,-150.000002,-53.964844,13n05011.abf,-3.000000
6,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,"(-67.767334, 3958)",0.124247,-100.000001,-53.474178,13n05011.abf,-2.000000
7,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,"(-62.493896, 1857)",0.521464,-50.000001,-55.415855,13n05011.abf,-1.000000
8,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,"(-48.66333, 4657)",1.362280,0.000000,-51.655174,13n05011.abf,0.000000
9,NaN,3.333889,0.26150,0.000000,0.09400,0.261500,0.261500,"(61.66382, 2275)",-11.669950,50.000001,-52.308094,13n05011.abf,1.000000


In [7]:
lsa_results['spikes_set'][12]

,threshold_index,clipped,threshold_t,threshold_v,threshold_i,peak_index,peak_t,peak_v,peak_i,trough_index,...,adp_index,adp_t,adp_v,adp_i,slow_trough_index,slow_trough_t,slow_trough_v,slow_trough_i,width,upstroke_downstroke_ratio
0,2711,False,0.13555,-41.107178,200.0,2735,0.13675,36.224365,200.0,2798,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00110,1.532822
1,3823,False,0.19115,-39.001465,200.0,3847,0.19235,37.078857,200.0,5057,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00125,1.596385
